In [2]:
from data_resolvers.mongo_db_connection import MongoDBConnection
from bson.objectid import ObjectId
from data_resolvers.ciselniky_data_resolver import CiselnikyDataResolver
from data_resolvers.nezrovnalost_data_resolver import NezrovnalostDataResolver

In [3]:
connection = MongoDBConnection('./appsettings.json')
client = connection.client

Pripájanie k databáze...
Pripojenie k databáze prebehlo úspešne!


In [4]:
client.list_database_names()
itms_db_name = 'itmsDB'
itmsDB = client.get_database(itms_db_name)
ciselniky_col_name = 'ciselniky'
ciselnikyDetail_col_name = 'ciselnikyDetail'
nezrovnalost_col_name = 'nezrovnalosti'
nezrovnalostDetail_col_name = 'nezrovnalostiDetail'

In [36]:
projektyUkonceneDetail_collection = itmsDB.get_collection('projektyUkonceneDetail')

In [37]:
projektyUkonceneDetail_collection.find_one({"id":1})

{'_id': ObjectId('659aa9f527fa10385a07d2a3'),
 'aktivity': [{'href': '/v2/aktivita/2', 'id': 2},
  {'href': '/v2/aktivita/1', 'id': 1}],
 'cisloZmluvy': 'Z312020A017',
 'createdAt': '2015-10-29T09:57:18.114Z',
 'datumKoncaHlavnychAktivit': '2019-12-01T00:00:00Z',
 'datumKoncaRealizacie': '2019-12-01T00:00:00Z',
 'datumPlatnostiZmluvy': '2015-11-05T00:00:00Z',
 'datumUcinnostiZmluvy': '2015-11-06T00:00:00Z',
 'datumZaciatkuHlavnychAktivit': '2015-08-01T00:00:00Z',
 'datumZaciatkuRealizacie': '2015-08-01T00:00:00Z',
 'dlzkaCelkovaHlavnychAktivit': 53,
 'dlzkaCelkovaProjektu': 53,
 'formyFinancovania': [{'hodnotaCiselnika': {'ciselnikKod': 1037,
    'href': '/v2/hodnotaCiselnika/1037/hodnota/1',
    'id': 1},
   'konkretnyCiel': {'href': '/v2/konkretnyCiel/8', 'id': 8}}],
 'hospodarskeCinnosti': [{'hodnotaCiselnika': {'ciselnikKod': 1038,
    'href': '/v2/hodnotaCiselnika/1038/hodnota/24',
    'id': 24},
   'konkretnyCiel': {'href': '/v2/konkretnyCiel/8', 'id': 8}}],
 'href': '/v2/projekt

In [38]:
all_intenzity_ids = projektyUkonceneDetail_collection.aggregate([
    {
        '$project': {
            'intenzity': {
                '$map': {
                    'input': '$intenzity', 
                    'as': 'intenzita', 
                    'in': '$$intenzita.id'
                }
            }
        }
    }, {
        '$unwind': {
            'path': '$intenzity'
        }
    }, {
        '$project': {
            '_id': 0
        }
    }
])

In [40]:
all_intenzity_ids_list = list(all_intenzity_ids)
# print(len(all_intenzity_ids_list))
print(all_intenzity_ids_list[2])
# for tmp in all_intenzity_ids:
#     print(tmp)

IndexError: list index out of range

In [ ]:
# ciselniky_data_res = CiselnikyDataResolver(connection, itms_db_name, 'ciselniky')
# ciselniky_data_res.fetch_remote_data()

# ciselnikyDetail_data_res = CiselnikyDetailDataResolver(
#     connection, 
#     itms_db_name,
#     ciselniky_col_name,
#     ciselnikyDetail_col_name)
# await ciselnikyDetail_data_res.fetch_remote_data_async()

# nezrovnalost_data_res = NezrovnalostDataResolver(connection, itms_db_name, nezrovnalost_col_name)
# nezrovnalost_data_res.fetch_remote_data()

nezrovnalostDetail_data_res = NezrovnalostDetailDataResolver(
    connection, 
    itms_db_name,
    nezrovnalost_col_name,
    nezrovnalostDetail_col_name)
remote_data = await nezrovnalostDetail_data_res.fetch_remote_data_async()

In [ ]:
def get_all_local_data():
    local_ciselnikDetail = itmsDB.get_collection(ciselnikyDetail_col_name).aggregate([
        {
            '$addFields': {
                'origin': '$$CURRENT', 
                'dict_key': {
                    '$concat': [
                        {
                            '$toString': '$ciselnikKod'
                        }, '_', {
                            '$toString': '$id'
                        }
                    ]
                }
            }
        }, {
            '$project': {
                'origin._id': 0
            }
        }, {
            '$project': {
                '_id': 1, 
                'origin': 1, 
                'dict_key': 1
            }
        }
    ])
    return {item['dict_key']:item for item in local_ciselnikDetail}

In [ ]:
nezrovnaost_url_template = 'https://opendata.itms2014.sk/v2/nezrovnalost?minId={minId}'

In [ ]:
all_ids = itmsDB.get_collection(nezrovnalost_col_name).distinct("id")